This notebook contains script that instantiates 'labelImg' window for creating bbox and convert those for retinnanet algorithm to consume. Note that image size is assumed to be 640 x 640

In [1]:
import xml.etree.ElementTree as ET
import numpy as np
import glob
import os
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
from six import BytesIO

2023-04-07 22:21:07.648288: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#start label img
# !important: while labelling ensure that labels are numerical, starting from 1. 1,2,3,..... etc.
#as the code is designed to handle only the numerals.
os.system("labelimg")

2023-04-07 22:22:43.559 Python[3615:238276] Warning: Window move completed without beginning


Image:/Users/ml/Desktop/ILS/object_detection/data/test2.jpg -> Annotation:/Users/ml/Desktop/ILS/object_detection/data/test2.xml


0

In [3]:
#xml stored in following directory
data_dir = "..//data"

In [4]:
#get all the xml file names generated after labelling from previous step
file_names = []
os.chdir(data_dir)
for file in glob.glob("*.xml"):
    file_names.append(file)

In [5]:
#convert xml file to format to be consumed by retina net algorithm

bbox_formatted_all = []
name_obj_all = []
for file_name in file_names:
    
    tree = ET.parse(file_name)
    
    root = tree.getroot()

    bbox_formatted= []
    name_obj_formatted= []
    obj_bbox = root.findall('./object')
    name_obj = []
    for item in obj_bbox:
        bbox = []
        name_obj.append(item.find('name').text)
        bbox_cord= item.find('./bndbox')
        bbox.append(
            [
            bbox_cord.find("ymin").text,
            bbox_cord.find("xmin").text,
            bbox_cord.find("ymax").text,
            bbox_cord.find("xmax").text
            ]  
        )
        bbox_formatted.append((bbox[0]))
    bbox_formatted_all.append(np.array(bbox_formatted, dtype = np.float32)/640.0) #640 is both and width dimention.
    name_obj_all.append(np.array(name_obj).astype(int))


In [6]:
#save bbox as .npy file
bbox_fn = 'bbox.npy'
with open(bbox_fn, 'wb') as f:
    np.save(f, bbox_formatted_all)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/lib/npyio.py:521: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [9]:
#save index of the class list
indices_class_fn = 'indices_class_list.npy'

name_obj_all = np.array(name_obj_all).astype(object)
with open(indices_class_fn, 'wb') as f:
    np.save(f, name_obj_all)